In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers

# Define the generator model
def build_generator(latent_dim, input_shape):
    model = tf.keras.Sequential()

    # Add LSTM layer
    model.add(layers.LSTM(256, input_shape=latent_dim[1:], return_sequences=True))
    model.add(layers.BatchNormalization())

    # Output layer
    model.add(layers.Conv3D(input_shape[-1], kernel_size=(3, 3, 3), activation='sigmoid', padding='same'))

    return model
'''
    # Add ConvLSTM2D layer
    model.add(layers.ConvLSTM2D(128, kernel_size=(3, 3), padding='same', return_sequences=True))
    model.add(layers.BatchNormalization())

    # Add Conv3D layer
    model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), padding='same'))
    model.add(layers.BatchNormalization())

    # Add Conv3DTranspose layer
    model.add(layers.Conv3DTranspose(32, kernel_size=(3, 3, 3), padding='same'))
    model.add(layers.BatchNormalization())
'''




# Define the discriminator model
def build_discriminator(input_shape):
    model = tf.keras.Sequential()

    # Add Conv3D layer
    model.add(layers.Conv3D(32, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same', input_shape=input_shape))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # Add Conv3D layer
    model.add(layers.Conv3D(64, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # Add Conv3D layer
    model.add(layers.Conv3D(128, kernel_size=(3, 3, 3), strides=(2, 2, 2), padding='same'))
    model.add(layers.LeakyReLU(alpha=0.2))
    model.add(layers.Dropout(0.25))

    # Flatten the output
    model.add(layers.Flatten())

    # Output layer
    model.add(layers.Dense(1, activation='sigmoid'))

    return model

# Define the combined GAN model
def build_gan(generator, discriminator):
    discriminator.trainable = False

    model = tf.keras.Sequential()
    model.add(generator)
    model.add(discriminator)

    return model

# Define the GAN training loop
def train_gan(generator, discriminator, gan, data, epochs, batch_size):
    for epoch in range(epochs):
        # Select a random batch of real samples
        idx = np.random.randint(0, data.shape[0], batch_size)
        real_samples = data[idx]

        # Generate a batch of fake samples
        latent_space = np.random.normal(0, 1, (batch_size, latent_dim))
        fake_samples = generator.predict(latent_space)

        # Concatenate real and fake samples
        all_samples = np.concatenate([real_samples, fake_samples])

        # Labels for real and fake samples
        labels = np.concatenate([np.ones((batch_size, 1)), np.zeros((batch_size, 1))])

        # Train the discriminator
        discriminator_loss = discriminator.train_on_batch(all_samples, labels)

        # Generate a new batch of latent space samples
        latent_space = np.random.normal(0, 1, (batch_size, latent_dim))

        # Labels for fake samples (pretend they are real)
        misleading_labels = np.ones((batch_size, 1))

        # Train the generator (via the GAN model, where the discriminator is frozen)
        gan_loss = gan.train_on_batch(latent_space, misleading_labels)

        # Print the progress
        print(f"Epoch: {epoch+1}/{epochs}, Discriminator Loss: {discriminator_loss}, GAN Loss: {gan_loss}")
timesteps, height, width, channels=10,1,1,4
# Define the input shape and latent dimension
input_shape = (timesteps, height, width, channels)
latent_dim = (32,1,1)

# Build the generator
generator = build_generator(latent_dim, input_shape)

# Build the discriminator
discriminator = build_discriminator(input_shape)

# Build the GAN
gan = build_gan(generator, discriminator)

# Compile the discriminator
discriminator.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')

# Compile the GAN
gan.compile(optimizer=tf.keras.optimizers.Adam(0.0002, 0.5), loss='binary_crossentropy')

# Train the GAN
train_gan(generator, discriminator, gan, data, epochs=200, batch_size=32)


ValueError: ignored